In [3]:
import json
import sys
import requests
import pandas as pd
from bs4 import BeautifulSoup


def pyynto(url):
    try:
        vastaus = requests.get(url, timeout=1000)
        vastaus.raise_for_status()
        return vastaus.content
    except requests.RequestException as e:
        print(f"Virhe datan haussa: {e}")
        sys.exit(1)

def keitto(html):
    try:
        html_str = html.decode('utf-8')
        soup = BeautifulSoup(html_str,'html.parser')
        data = json.loads(soup.find('script', id='online-match-data').text)
        return(data)
    except AttributeError as e:
        print(f'Virhe: {e}')
        sys.exit(1)

def kokoonpano(data):
    
    kotikokoopano = pd.json_normalize(data['teams']['home']["lineUp"])
    
    vieraskokoopano = pd.json_normalize(data['teams']['away']["lineUp"])
    
    kotipj = pd.json_normalize(
        data['teams']['home']["managersLineUp"])
    
    vieraspj = pd.json_normalize(
        data['teams']['away']["managersLineUp"])
    
    kokoonpano = pd.concat([kotikokoopano, vieraskokoopano],  ignore_index=False)

def data_muokkaus(data):
    try:

        koti = pd.json_normalize(data['teams']['home'], record_path=['lineUp'],
                                 meta=['id', 'name', 'shorthand', 'icon', 'logo'])

        koti_pelinjohtajat = pd.json_normalize(
            data['teams']['home'], record_path=['managersLineUp'])

        koti_kaarten_väli = pd.json_normalize(data['details'], record_path=[
                                              "innerCircle_home"], meta=['id'], record_prefix='sisäkaari_', errors='ignore')

        kotivalmennus = pd.json_normalize(data['details'], record_path=[
                                          "OtherRoles:home"], record_prefix='valmennus', errors='ignore')

        kotivalmennus.columns = ['valmennuskoti']

        vieras = pd.json_normalize(data['teams']['away'], record_path=['lineUp'],
                                   meta=['id', 'name', 'shorthand', 'icon', 'logo'])

        vieras_pelinjohtajat = pd.json_normalize(
            data['teams']['away'], record_path=['managersLineUp'])

        vieras_kaarten_väli = pd.json_normalize(data['details'], record_path=[
                                                "innerCircle_away"], meta=['id'], record_prefix='sisäkaari_', errors='ignore')

        vierasvalmennus = pd.json_normalize(data['details'], record_path=[
                                            "OtherRoles:away"], record_prefix='valmennus', errors='ignore')

        vierasvalmennus.columns = ['valmennusvieras']

        kotijoukkue_pelinjohto = pd.merge(
            koti_pelinjohtajat, koti_kaarten_väli, left_on='manager.id', right_on='sisäkaari_id', how='outer')

        vierasjoukkue_pelinjohto = pd.merge(
            vieras_pelinjohtajat, vieras_kaarten_väli, left_on='manager.id', right_on='sisäkaari_id', how='outer')

        koti_kotiutuslyönti = pd.json_normalize(
            data['details']["scoringContestPlayers:0"], record_path=['home'], record_prefix='koti_')

        vieras_kotiutuslyönti = pd.json_normalize(
            data['details']["scoringContestPlayers:0"], record_path=['away'], record_prefix='vieras_')

        koti_kotiutuslyöntijatkot = pd.json_normalize(
            data['details']["scoringContestPlayers:1"], record_path=['home'], record_prefix='kotijatko_')

        vieras_kotiutuslyöntijatkot = pd.json_normalize(
            data['details']["scoringContestPlayers:1"], record_path=['away'], record_prefix='vierasjatko_')

        muut_tiedot = pd.json_normalize(data['details'], errors='ignore')

        kokoonpano = pd.concat([koti, vieras],  ignore_index=False)

        kotarit = pd.concat([koti_kotiutuslyönti, vieras_kotiutuslyönti, koti_kotiutuslyöntijatkot,
                            vieras_kotiutuslyöntijatkot],  axis=1, ignore_index=False)

        pelinjohto = pd.concat(
            [kotijoukkue_pelinjohto, vierasjoukkue_pelinjohto], ignore_index=False)

        valmennus = pd.concat([kotivalmennus, vierasvalmennus],
                              axis=1, join='outer', ignore_index=False)

        return (kokoonpano, kotarit, pelinjohto, valmennus, muut_tiedot)
    except (TypeError, KeyError) as e:
        print(f'Virhe {e}')


def main():

    ottelunUrl = ('https://v1.pesistulokset.fi/media/ottelu/83977')

    hae_data = pyynto(ottelunUrl)

    tiedot = keitto(hae_data)
    
    global data
    
    data = tiedot
    
    

main()

In [4]:
data

{'id': 83977,
 'teams': {'home': {'id': 12493,
   'name': 'Vimpelin Veto',
   'shorthand': 'ViVe',
   'icon': 'https://www.pesistulokset.fi/storage/seuralogot/2a5c10a7d245d83f666f5422aae63628-icon.png',
   'logo': 'https://www.pesistulokset.fi/storage/seuralogot/2a5c10a7d245d83f666f5422aae63628.png',
   'lineUp': [{'player': {'id': 12047,
      'firstName': 'Olli',
      'lastName': 'Heikkala',
      'url': 'https://v1.pesistulokset.fi/pelaaja/12047',
      'mainId': 12047,
      'image': 'https://www.pesistulokset.fi/storage/pelaajakuvat/18bd2573dcdaf43ee45e0b6e0f96861c-maxWidth125.png',
      'age': 29,
      'birthday': '23.12.1995',
      'birthday_today': False},
     'originalNumber': 1,
     'defensivePosition': 3,
     'prizes': [],
     'trophies': [{'id': 40,
       'name': 'SM-hopeamitali',
       'detail': None,
       'season': 2023},
      {'id': 39, 'name': 'Suomen mestaruus', 'detail': None, 'season': 2022},
      {'id': 55,
       'name': 'Miesten Itä-Länsi-ottelu',
  

In [9]:
kotikokoopano = pd.json_normalize(data['teams']['home']["managersLineUp"])

In [107]:
kotikokoopano["nimi"] = data['teams']['home']["shorthand"]

In [69]:
koti = data["details"]["scoringContestPlayers:0"]["home"]

In [38]:
vieras = data["details"]["scoringContestPlayers:0"]["away"]

In [39]:
vierasjatko = data["details"]["scoringContestPlayers:1"]["away"]

In [33]:
kotijatko = data["details"]["scoringContestPlayers:1"]["koti"]

In [18]:
vieras = data["details"]["innerCircle_away"]

In [22]:
vaihtopenkki = {"koti": data["details"]["OtherRoles:home"],
                "vieras": data["details"]["OtherRoles:away"]}

In [20]:
rooli = pd.concat([pd.json_normalize(koti), pd.json_normalize(vieras)])

In [40]:
kotari = pd.concat([pd.json_normalize(koti), pd.json_normalize(kotijatko), pd.json_normalize(vieras), pd.json_normalize(vierasjatko)])

In [67]:
kotikotari = pd.concat([pd.json_normalize(koti), pd.json_normalize(
    kotijatko)])

In [97]:
vieraskotari = pd.concat([pd.json_normalize(vieras), pd.json_normalize(
    vierasjatko)])

In [96]:
kotikokoopano = pd.json_normalize(data['teams']['home']["lineUp"])
    
vieraskokoopano = pd.json_normalize(data['teams']['away']["lineUp"])
    
kotipj = pd.json_normalize(
        data['teams']['home']["managersLineUp"])
    
vieraspj = pd.json_normalize(
        data['teams']['away']["managersLineUp"])
    
kokoonpano = pd.concat([kotikokoopano, vieraskokoopano],  ignore_index=False)

In [74]:
kotikotaril = vieraskotari[["runner", "batter"]]

In [98]:
vieraskotari = vieraskotari[["runner", "batter"]]

In [109]:
kotikokoopano = pd.json_normalize(data['teams']['home']["lineUp"])

vieraskokoopano = pd.json_normalize(data['teams']['away']["lineUp"])

kotikokoopano["joukkue"] = data['teams']['home']["shorthand"]

vieraskokoopano["joukkue"] = data['teams']['away']["shorthand"]
    
kotikokoopano["etenijänimi"] = kotikokoopano["player.firstName"] + \
        " " + kotikokoopano["player.lastName"]
        
kotikokoopano["lyöjänimi"] = kotikokoopano["player.firstName"] + \
        " " + kotikokoopano["player.lastName"]
        
vieraskokoopano["etenijänimi"] = vieraskokoopano["player.firstName"] + \
        " " + vieraskokoopano["player.lastName"]

vieraskokoopano["lyöjänimi"] = vieraskokoopano["player.firstName"] + \
        " " + vieraskokoopano["player.lastName"]
    
kotieka = data["details"]["scoringContestPlayers:0"]["home"]
    
vieraseka = data["details"]["scoringContestPlayers:0"]["away"]
    
kotijatko = data["details"]["scoringContestPlayers:1"]["home"]
    
vierasjatko = data["details"]["scoringContestPlayers:1"]["away"]
    
kotikotari = pd.concat([pd.json_normalize(kotieka), pd.json_normalize(
        kotijatko)])
    
vieraskotari = pd.concat([pd.json_normalize(vieraseka), pd.json_normalize(
        vierasjatko)])
    
kotietenijakotari = kotikotari.merge(kotikokoopano[[
        "etenijänimi", "joukkue", "originalNumber"]], left_on="runner", right_on="originalNumber")
    
kotikotarilopullinen = kotietenijakotari.merge(kotikokoopano[[
    "lyöjänimi", "joukkue", "originalNumber"]], left_on="batter", right_on="originalNumber", suffixes=("_etenijä", "_lyöja"))
    
vierasetenijakotari = vieraskotari.merge(vieraskokoopano[[
    "etenijänimi", "joukkue", "originalNumber"]], left_on="runner", right_on="originalNumber")

vieraskotarilopullinen = vierasetenijakotari.merge(vieraskokoopano[[
    "lyöjänimi", "joukkue", "originalNumber"]], left_on="batter", right_on="originalNumber", suffixes=("_etenijä", "_lyöja"))
    
pd.concat([kotikotarilopullinen, vieraskotarilopullinen])

,batter,runner,etenijänimi,joukkue_etenijä,originalNumber_etenijä,lyöjänimi,joukkue_lyöja,originalNumber_lyöja
0,3,11,Jere Saukko,ViVe,11,Elmeri Anttila,ViVe,3
1,10,1,Olli Heikkala,ViVe,1,Jukka-Pekka Vainionpää,ViVe,10
2,4,7,Juho Heikkala,ViVe,7,Matias Rinta-aho,ViVe,4
3,12,6,Mikko Kanala,ViVe,6,Janne Mäkelä,ViVe,12
4,2,5,Mikko Vihriälä,ViVe,5,Aleksi Lassila,ViVe,2
5,10,1,Olli Heikkala,ViVe,1,Jukka-Pekka Vainionpää,ViVe,10
6,4,5,Mikko Vihriälä,ViVe,5,Matias Rinta-aho,ViVe,4
7,12,11,Jere Saukko,ViVe,11,Janne Mäkelä,ViVe,12
0,11,1,Konsta Kettunen,JoMa,1,Juho Toivola,JoMa,11
1,3,5,Kasperi Hämäläinen,JoMa,5,Niko Korhonen,JoMa,3


In [ ]:
"""query TulosvetoDrawDetailsQuery($drawId: ID!, $lang: String) {  scoreDraws(ids: [$drawId], lang: $lang) {    id    betSlipId    closeTime    row {      name      id      eventId      sportId      outcome {        home        away      }      score {        min        max      }      event {        tournament {          name          id        }        externalId      }    }    popularity {      exchange      home {        score        percentage      }      away {        score        percentage      }    }    gameRuleSet {      oddsType      minStake      maxStake      maxSelections      maxPrice    }    resultsAvailableTime    jackpots {      type    }  }  tulosvetoOdds(drawId: $drawId) {    timestamp    exchange    odds {      value      baseValue      selections {        homeScores        awayScores      }    }  }}
"""